In [1]:
# Install requirements

!pip install llama-index
!pip install sentence-transformers
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index-readers-json
!pip install llama-index-vector-stores-milvus
!pip install pymilvus
!pip install accelerate
!pip install bitsandbytes
!pip install llama-index-llms-groq
!pip install FlagEmbedding
!pip install peft
!pip install datasets
!pip install transformers
!pip install loralib
!pip install einops
!pip install huggingface_hub
!pip install python-dotenv

!pip install flask-ngrok
!pip install pyngrok


from IPython.display import clear_output
clear_output()

In [2]:
# Model configurations
import os
from dotenv import load_dotenv
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from huggingface_hub.hf_api import HfFolder

REPO_ACCESS_TOKEN = os.getenv("REPO_ACCESS_TOKEN")
MODEL_NAME = "Namronaldo2004/math-vinallama-7b-chat"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config = bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
model.generation_config = generation_config

adapter_config.json:   0%|          | 0.00/796 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/322 [00:00<?, ?B/s]

In [3]:
# Chatbot
def generate_response(query: str, history: dict[str, str]):
    prompt = f"""
    <|im_start|>system
    Bạn là một chuyên gia về toán. Khi nhận được yêu cầu từ người dùng, dựa vào những kiến thức mà bạn đã có, hãy trả lời người dùng một cách nhất quán, đầy đủ nhưng tránh dư thừa thông tin, và hơn hết nội dung câu trả lời phải chính xác nhất
    <|im_end|>
    <|im_start|>user
    ### Yêu cầu:
    {query}
    ### Câu trả lời:
    <|im_end|>
    <|im_start|>assistant
    """.strip()

    init_msg = {
        "role": "system",
        "content": "Bạn là một chuyên gia về toán. Khi nhận được yêu cầu từ người dùng, dựa vào những kiến thức mà bạn đã có, hãy trả lời người dùng một cách nhất quán, đầy đủ nhưng tránh dư thừa thông tin, và hơn hết nội dung câu trả lời phải chính xác nhất"
    }
        
    history = [init_msg, *history, {"role": "user", "content": query}]
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    encoding = tokenizer(prompt, return_tensors = "pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids = encoding.input_ids,
            attention_mask = encoding.attention_mask,
            generation_config = generation_config
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens = True).split("<|im_start|> assistant")[1].strip()

In [4]:
# Expose api with ngrok

!ngrok config add-authtoken 2r1MdG5rU3fQL4aoO92QFztrerN_ejwgH3dUcde6eiEELLgC

from pyngrok import ngrok
import os

# Create public URL for the app
public_url = ngrok.connect(5000)
print("Service is live at:", public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                
Service is live at: NgrokTunnel: "https://69f1-34-44-191-114.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
# Flask app

from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/")
def home():
    return jsonify({ "message": "Service is alive" }), 200

@app.route("/chat", methods=["POST"])
def chat():
  data = request.get_json()
  reqMsg = data.get("message", None)
  reqHistory = data.get("history", None)
  if not reqMsg or reqHistory == None: return jsonify({ "message": "Bad request"}), 400
  try:
    resMsg = generate_response(reqMsg, reqHistory)
    return jsonify({ "message": resMsg }), 200
  except:
    return jsonify({ "message": "Server internal error"}), 500

if __name__ == "__main__":
  app.run()


 * Serving Flask app '__main__'
 * Debug mode: off
